In [1]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
data = pd.read_csv(r"C:\Users\alrem\Desktop\données EURUSD\data_elonmusk.csv", encoding='latin1')

In [ ]:
data

In [4]:
tweets = data["Tweet"]

In [ ]:
tweets

In [ ]:
nltk.download('vader_lexicon')
analyzer = SentimentIntensityAnalyzer()

In [7]:
def analyze_sentiment(tweet):
    sentiment = analyzer.polarity_scores(tweet)
    compound_score = sentiment['compound']
    if compound_score >= 0.5:
        return "Positive"
    elif compound_score <= -0.5:
        return "Negative"
    else:
        return "Neutral"

In [8]:
tesla_tweets = data[data['Tweet'].str.lower().str.contains("tesla","TSLA")]

In [ ]:
tesla_tweets['Sentiment'] = tesla_tweets['Tweet'].apply(analyze_sentiment)

In [ ]:
print(tesla_tweets[['Tweet', 'Sentiment']])

In [13]:
# Initialisation des variables
current_position = "None"  # Au début, aucune position n'est ouverte
cash_balance = 10000.0  # Solde de trésorerie initial
portfolio = {}  # Dictionnaire pour suivre les positions ouvertes



# Logique de trading
for i in range(len(tesla_tweets)):
    tweet_sentiment = tesla_tweets.iloc[i]['Sentiment'] # Avec la fonction iloc, il est possible de récupérer toutes les données des variables à cette date
    
    # Simulez ici la logique d'achat et de vente en fonction du sentiment
    if tweet_sentiment == "Positive":
        if current_position == "None" and cash_balance > 0:
            # Achat de Tesla
            position_size = 0.2 * cash_balance  # Exemple : investir 20% du solde en TSLA
            cash_balance -= position_size # Cela soustrait le montant de la position achetée (position_size) du solde de trésorerie actuel
            portfolio["TSLA"] = position_size # Cela ajoute la position achetée dans le dictionnaire portfolio en utilisant le symbole de l'action "TSLA" comme clé et la taille de la position (position_size) comme valeur. 
            current_position = "Long"
            print("Achat de Tesla")
    elif tweet_sentiment == "Negative":
        if current_position == "Long" and "TSLA" in portfolio:
            # Vente de Tesla
            position_size = portfolio["TSLA"]
            cash_balance += position_size
            del portfolio["TSLA"]
            current_position = "None"
            print("Vente de Tesla")

# Afficher le solde final du portefeuille
print("Solde final du portefeuille :", cash_balance)

In [ ]:
# Importez les bibliothèques nécessaires de QuantConnect
from AlgorithmImports import *
from AlphaModelImports import *

class SentimentBasedTradingAlgorithm(QCAlgorithm):
    def Initialize(self):
        self.SetCash(100000)  # Solde de départ en USD
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 1, 31)
        
        # Ajoutez le symbole de l'action Tesla
        self.tesla = self.AddEquity("TSLA")
        
        # Créez un résolveur de sentiment (à adapter en fonction de votre source de données)
        self.sentiment_resolver = SentimentResolver()
        
        # Planifiez la fonction de trading pour chaque jour de trading
        self.Schedule.On(self.DateRules.EveryDay("TSLA"), self.TimeRules.AfterMarketOpen("TSLA"), self.Trade)

    def Trade(self):
        # Obtenez le sentiment actuel des tweets sur Tesla (à adapter en fonction de votre résolveur de sentiment)
        tweet_sentiment = self.sentiment_resolver.analyze_sentiment()
        
        # Si le sentiment est positif, achetez des actions Tesla
        if tweet_sentiment == "Positive":
            self.SetHoldings("TSLA", 0.5)  # Investissez 50 % du capital dans Tesla
        # Si le sentiment est négatif, vendez des actions Tesla
        elif tweet_sentiment == "Negative":
            self.SetHoldings("TSLA", 0)  # Liquidation de la position
        
        # Affichez le sentiment et les positions
        self.Debug(f"Sentiment: {tweet_sentiment}")
        self.Debug(f"Portfolio Holdings: {self.Portfolio['TSLA'].Quantity}")

# Classe SentimentResolver (à adapter en fonction de votre source de données)
class SentimentResolver:
    def analyze_sentiment(tweet):
        sentiment = analyzer.polarity_scores(tweet)
        compound_score = sentiment['compound']
        if compound_score >= 0.5:
            return "Positive"
        elif compound_score <= -0.5:
            return "Negative"
        else:
            return "Neutral"
        
# Lancez l'algorithme
from AlgorithmImports import *
from AlphaModelImports import *
SentimentBasedTradingAlgorithm()